In [41]:
import pandas as pd

df = pd.read_csv("data/preprocessed_data.csv")

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
df.head(5)

,Politikbereich,Zweck
0,Verkehr,"Nord-Süd-Tangente; Linie 26/27, 2.2. Teil-BPU,A"
1,Bildung,Gedenken zu 30 Jahre Mauerfall
2,Stadtentwicklung,Lernen Na Logo - Bildungsnetzwerk Hellersdorfer Promenade
3,Jugend,Kinder- und Jugendambulanz
4,Jugend,Therapiebad


In [42]:
X, y = df.loc[:, df.columns != 'Politikbereich'], df.loc[:,df.columns == 'Politikbereich']

In [43]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

le.fit(y["Politikbereich"].unique().tolist())
y = y["Politikbereich"].apply(lambda s: le.transform([s])[0]).values
y

array([21,  2, 18, ..., 20, 20, 20])

In [45]:
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import spacy
import torch
from transformers import BertModel, BertTokenizer
from lib.bert_pytorch.helper_functions import get_device
import nlpaug.augmenter.word as naw

class SelectFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, features_list):
        self.features_list = features_list
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        X = X[self.features_list]
        X = X.iloc[:,0]
        return X

class CleanText(BaseEstimator, TransformerMixin):
    def cleaner(self, text):
        # Remove mid slash and digits
        text = re.sub(r'-', ' ', text)
        text = re.sub(r'\d+', '', text)
        # Custom ones not supported by spacy
        text = re.sub(r'Abs\.', 'Absatz', text)
        text = re.sub(r'e\.V\.', 'eingetragener Verein', text)
        text = re.sub(r'co\.', 'Kompanie', text)
        text = re.sub(r'Co\.', 'Kompanie', text)
        text = re.sub(r'gem\.', 'gemäß', text)
        text = re.sub(r"'s", '', text)
        return text
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        X = X.apply(self.cleaner)
        return X

class SpacyLemmatizer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.nlp = spacy.load("de_core_news_lg")
        self.nlp.remove_pipe("ner")
        self.nlp.remove_pipe("parser")
        self.nlp.remove_pipe("attribute_ruler")
    def normalize(self, text):
        doc = self.nlp(text)
        output = []
        for token in doc:
            if not token.is_punct and not token.is_stop and not token.is_space:
                output.append(token.lemma_)
        return " ".join(output)
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        X = X.apply(self.normalize)
        return X

class Lowercase(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        X = X.apply(lambda text: text.lower())
        return X

class TfIdfVectorizer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.vectorizer = TfidfVectorizer()
    def fit(self, X, y=None):
        self.vectorizer.fit(X)
        return self
    def transform(self, X, y=None):
        tfidf_encodings = self.vectorizer.transform(X)
        X = pd.DataFrame(tfidf_encodings.toarray())
        return X

class BertSentenceEmbeddigs(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-german-cased')
        self.device = get_device()
        self.model = BertModel.from_pretrained('bert-base-german-cased', 
                                                output_hidden_states=True)\
                                                    .to(self.device)
        self.model.eval()
    def embed_sentence(self, sentence: str):
        ids_tensor = self.tokenizer.encode(sentence, return_tensors='pt')
        ids_tensor = ids_tensor.to(self.device)
        with torch.no_grad():
            out = self.model(input_ids=ids_tensor)
        hidden_states = out.hidden_states
        last_four_layers = [hidden_states[i] for i in (-1, -2, -3, -4)]
        sentence_embedding = torch.cat(tuple(last_four_layers), dim=0)
        sentence_embedding = torch.mean(sentence_embedding, dim=0)
        sentence_embedding = torch.mean(sentence_embedding, dim=0)
        return sentence_embedding.cpu().numpy()
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        X = X.apply(self.embed_sentence)
        X = pd.DataFrame(X.values.tolist())
        return X

prep_pipeline = Pipeline([
    ("select_features", SelectFeatures(features_list=["Zweck"])),
    # ("clean_text", CleanText()),
    # ("spacy_lemmatizer", SpacyLemmatizer()),
    # ("text_lowercase", Lowercase()),
    # ("tfidf_vectorizer", TfIdfVectorizer()),
    ("bert_sentence_embeddings", BertSentenceEmbeddigs()),
])

# Example
prep_pipeline.fit_transform(X, y).head(3)

There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 2070 with Max-Q Design


Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.680419,0.433664,0.439248,0.866302,-0.352283,0.405156,0.042996,0.474451,-0.042971,-0.087204,...,0.303186,-0.299498,0.331566,0.211172,0.274229,0.532789,-0.644731,0.511696,0.032847,0.031498
1,0.066066,-0.440251,-0.327377,0.168775,-0.177032,0.491001,-0.455673,0.249439,-0.828507,0.019433,...,-0.320476,-0.197653,0.019215,-0.156263,-0.743738,0.104028,-0.065598,0.303763,0.365776,0.199047
2,0.026205,-0.194637,0.063422,0.138446,-0.160203,0.162052,-0.022508,0.377391,-0.206134,0.017674,...,-0.227518,0.198408,0.382017,-0.227325,0.116455,0.193815,-0.478202,0.141823,0.456079,-0.318012


### Parameters hypertuning and model selection with GridSearchCV

In [46]:
import os
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, make_scorer


def custom_scorer_macro_f1(y_true,y_pred):
    macro_f1 = f1_score(y_true, y_pred, average='macro')
    return macro_f1

scorer_macro_f1 = make_scorer(custom_scorer_macro_f1, greater_is_better=True)

def custom_scorer_weighted_f1(y_true,y_pred):
    weighted_f1 = f1_score(y_true, y_pred, average='weighted')
    return weighted_f1

scorer_weighted_f1 = make_scorer(custom_scorer_weighted_f1, greater_is_better=True)


def execute_pipeline(features,labels, search_space=[
                    {"estimator": [RandomForestClassifier(random_state=42, verbose=1)],
                    "estimator__n_estimators": [100],
                    # "estimator__max_depth": [2, 6]
                    }], 
                    cv = 10,
                    verbose = 1,
                    n_jobs = os.cpu_count() - 2,
                    scoring = scorer_macro_f1):
                    # scoring = {"macro_f1": scorer_macro_f1,"weighted_f1": scorer_weighted_f1}):
    
    pipe = Pipeline([("preprocessing", prep_pipeline),
                    ("estimator", RandomForestClassifier())])
    
    gridsearch = GridSearchCV(pipe, search_space, scoring=scoring, cv=cv, verbose=verbose,n_jobs=n_jobs,refit=True)
    best_model = gridsearch.fit(features, labels)
    print(best_model.best_params_)
    print(best_model.best_score_)
    return best_model

best_estimator = execute_pipeline(X, y)

There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 2070 with Max-Q Design


Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Fitting 10 folds for each of 1 candidates, totalling 10 fits
There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 2070 with Max-Q Design


Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 2070 with Max-Q Design


...

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 2070 with Max-Q Design


Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 2070 with Max-Q Design


Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [35]:
import pickle

pickle.dump(best_estimator,open("pretrained_models/random_forest/best_estimator.pkl", "wb" ))